# Head

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/MambaVAE")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Mounted at /content/gdrive


In [ ]:
%pip install datasets

In [ ]:
%pip install causal_conv1d

# vae

In [ ]:
import torch, InitMamba, importlib
from datasets import load_from_disk
importlib.reload(InitMamba)
import VAE
importlib.reload(VAE)
from VAE import MambaVAE
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer
from data.Dataset import get_dataset, tokenizer

vae = MambaVAE().cuda().eval()
vae.load_state_dict(torch.load('./results/vae/model.pth', weights_only=True), strict=False)

dataset = load_from_disk("./data/CoT3")
# train_dataset, eval_dataset = get_dataset()

In [ ]:
id = 1
question_ids = torch.tensor([dataset[id]['question_ids']], device='cuda')
question_mask = torch.tensor([dataset[id]['question_mask']], device='cuda')
full_ids = torch.tensor([dataset[id]['full_ids']], device='cuda')
full_mask = torch.tensor([dataset[id]['full_mask']], device='cuda')
full_loss_mask = torch.tensor([dataset[id]['full_loss_mask']], device='cuda')
tokenizer.batch_decode(full_ids, skip_special_tokens=True)[0]

In [ ]:
# input_ids, mask = question_ids, question_mask
# input_ids, mask = thoughts_ids, thoughts_mask
vae.eval()
state, log_var = vae.encode(full_ids, full_mask)

# idx = (state.abs() > 3)
# print(idx.sum() / state.numel())
# mix_state = state.clone()
# mix_state[idx] = torch.randn_like(state[idx])

res = vae.decode(state, full_ids, full_mask, full_loss_mask)
print(res.loss)
tokenizer.batch_decode(res.logits.argmax(-1), skip_special_tokens=True)

In [ ]:
r = 0.2
noise = torch.randn_like(state)
mix_state = (state*r**0.5+noise*(1-r)**0.5)

# idx = (state.abs() < 0.1)
# print(idx.sum() / state.numel())
# mix_state[idx] = torch.randn_like(state[idx]) * 0
vae.eval()
empty_ids = tokenizer('<|startoftext|>', return_tensors='pt').to('cuda')
tokens = vae.decoder.generate(input_ids = question_ids[:,:question_mask.sum()],
                inputs_ssm_states = vae.mlp2(mix_state),
                inputs_ssm_layer = 11,
                max_length = 256,
                # num_beams = 10
                )
tokenizer.batch_decode(tokens, skip_special_tokens=True)

In [ ]:
import matplotlib.pyplot as plt
vae.train()
data_np = states.flatten().detach().cpu().numpy()
plt.hist(data_np, bins=50)
plt.show()

In [ ]:
vae.sample(state, log_var)

# baseline


In [ ]:
import torch, transformers, os
from transformers import TrainingArguments, AutoTokenizer
from InitMamba import MambaForCausalLM
from datasets import load_dataset
import dataset.Dataset as Dataset

tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
model = MambaForCausalLM.from_pretrained('state-spaces/mamba-130m-hf').cuda()
dataset = load_dataset("arrow", data_files = './cache/CoT_full.arrow', split = 'train').select(range(100000))
print(dataset)
tot = len(dataset)
eval_size= int(tot * 0.05)
train_dataset = dataset.select(range(eval_size, tot))
eval_dataset = dataset.select(range(eval_size))
model.load_state_dict(torch.load("results/CoT_full/model.pth", weights_only=True))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
input_str = tokenizer.decode(eval_dataset[1]['input_ids'], skip_special_tokens=True)
input_ids = tokenizer(input_str.split(' Thinking: ')[0] + ' Thinking: ', return_tensors = 'pt')['input_ids'].cuda()
tokens = model.generate(
    input_ids,
    max_length = 512,
    # num_return_sequences = 5,
    # do_sample = False,
    # num_beams = 5
    )
print(input_str)
tokenizer.batch_decode(tokens)

# diffuser

In [ ]:
import importlib, torch
from datasets import load_from_disk
from VAE import MambaVAE
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer

dataset = load_from_disk("./data/CoT3")

tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
tokenizer.add_tokens(["<|startoftext|>", "<|thoughts|>", "<|answer|>"])

vae = MambaVAE().cuda().eval()
vae.load_state_dict(torch.load('./results/vae0.2/model.pth', weights_only=True))

In [ ]:
import Diffuser, importlib, torch
importlib.reload(Diffuser)
from Diffuser import Diffuser
from safetensors.torch import load_file

diff = Diffuser().cuda().eval()
diff.load_state_dict(load_file('results/diff46'))

In [ ]:
id = 1
question_ids = torch.tensor([dataset[id]['question_ids']], device='cuda')
question_mask = torch.tensor([dataset[id]['question_mask']], device='cuda')
full_ids = torch.tensor([dataset[id]['full_ids']], device='cuda')
full_mask = torch.tensor([dataset[id]['full_mask']], device='cuda')
full_loss_mask = torch.tensor([dataset[id]['full_loss_mask']], device='cuda')

target = vae.encode(full_ids, full_mask)[0].clamp(-1, 1)
condition = vae.encode(question_ids, question_mask)[0].clamp(-1, 1)

tokenizer.batch_decode(full_ids, skip_special_tokens=True)

In [ ]:
res = diff(target = target, condition = condition)
print(res)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(target.unsqueeze(-1).reshape(64, 64).detach().cpu())
plt.colorbar()
plt.show()

In [ ]:
# states = diff.generate(condition, 1000)
print((states - target).pow(2).mean())
vae.decode(states, full_ids, full_mask, full_loss_mask)

In [ ]:
import torch
import matplotlib.pyplot as plt

plt.imshow(states.reshape(64, 64).detach().cpu())
plt.colorbar()
plt.show()

In [ ]:
r = 0
tokens = vae.decoder.generate(input_ids = question_ids[:,:question_mask.sum()],
                inputs_ssm_states = vae.mlp2(torch.randn_like(states)*r**0.5 + states*(1-r)**0.5),
                inputs_ssm_layer = 11,
                max_length = 256,
                # do_sample=True,
                # num_return_sequences=3
                )
tokenizer.batch_decode(tokens, skip_special_tokens=True)

# Run

In [ ]:
!python data/Dataset.py

In [ ]:
!python VAE_trainer.py

In [ ]:
!python Baseline_trainer.py

In [ ]:
!python Diffuser_trainer.py

In [ ]:
from google.colab import runtime
runtime.unassign()